In [1]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
import numpy as np
import os
import glob
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

In [5]:
print(os.getcwd())
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
# print(chess_types_folders)
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

/Users/giangto/Documents/umn/csci5525/5525Chess


In [27]:
#sample df test set and df train set
chess_test = pd.DataFrame()
for i in range(1,7):
    chess_i = chess_df[chess_df["label"]==i].sample(frac=0.3)
    chess_test = pd.concat([chess_i,chess_test])
chess_test = chess_test[chess_df.columns]
print(chess_test.head())
chess_df = chess_df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)
chess_test = chess_test.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

# Merge
chess_train = chess_df.merge(chess_test, how='left', indicator=True)
chess_train = chess_train[chess_train['_merge'] == 'left_only'].drop(columns='_merge')


                                        normalized_img  label
747  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      6
828  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      6
804  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      6
736  (0.13725490196078433, 0.12941176470588237, 0.1...      6
794  (0.6470588235294118, 0.6352941176470588, 0.635...      6


/var/folders/tp/pqhy6s2j2lq2h7_jn7jz7gpr0000gn/T/ipykernel_5089/449878163.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chess_df = chess_df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)
/var/folders/tp/pqhy6s2j2lq2h7_jn7jz7gpr0000gn/T/ipykernel_5089/449878163.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chess_test = chess_test.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)


In [28]:
print("chess train labels: ", chess_train["label"].value_counts())
print("chess test labels: ", chess_test["label"].value_counts())


chess train labels:  label
5    109
2    109
4    106
3     95
6     90
1     88
Name: count, dtype: int64
chess test labels:  label
5    47
2    47
4    46
3    41
6    38
1    38
Name: count, dtype: int64


In [15]:
chess_test.describe()

,label
count,219.000000
mean,3.077626
std,1.407191
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [3]:
print(chess_df.head())

                                      normalized_img  label
0  [0.2196078431372549, 0.2235294117647059, 0.227...      3
1  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      3
2  [0.8666666666666667, 0.8666666666666667, 0.866...      3
3  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      3
4  [0.6862745098039216, 0.6823529411764706, 0.690...      3


In [4]:
chess_df.iloc[0:5]['normalized_img'].shape #dim of 1 single image

(5,)

In [5]:
chess_df.iloc[0]['normalized_img']

array([0.21960784, 0.22352941, 0.22745098, ..., 0.76078431, 0.76078431,
       0.76078431], shape=(50176,))

In [29]:
#Load dataset
X_train = np.array(chess_train['normalized_img'])
X_train = np.array([x for x in X_train])
y_train = np.array(chess_train['label'])

X_test = np.array(chess_test['normalized_img'])
X_test = np.array([x for x in X_test])
y_test = np.array(chess_test['label'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
#Selecting best C for Linear SVM Model
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		lin_svm = SVC(kernel='linear',C=C)
		lin_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = lin_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = lin_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)





C = 0.01: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 0.1: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 1.0: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 10: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [31]:
#Fit linear model on best C
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='linear',C=C_chosen)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"Linear SVM Performance Metrics with C={C_chosen}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 0.01, with lowest avg test error: 0.0000
Final Test Error: 0.8132
Linear SVM Performance Metrics with C=0.01:
Accuracy: 0.1868
Precision: 0.1845
Recall: 0.1868
F1 Score: 0.1846

Confusion Matrix:
[[ 6  6  4  6 10  6]
 [ 6 15  5  5 13  3]
 [ 7  8  5  5  6 10]
 [ 5  9  6 10  9  7]
 [ 4 10  9  9  5 10]
 [ 5  9  4  7  6  7]]

Classification Report:
              precision    recall  f1-score   support

           1       0.18      0.16      0.17        38
           2       0.26      0.32      0.29        47
           3       0.15      0.12      0.14        41
           4       0.24      0.22      0.23        46
           5       0.10      0.11      0.10        47
           6       0.16      0.18      0.17        38

    accuracy                           0.19       257
   macro avg       0.18      0.18      0.18       257
weighted avg       0.18      0.19      0.18       257



In [32]:
#Selecting best C for RBF SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10,100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='rbf',C=C, gamma='auto')
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)


C = 0.01: (avg train error, avg validation error) = (0.7722, 0.7722)
C = 0.1: (avg train error, avg validation error) = (0.7722, 0.7722)
C = 1.0: (avg train error, avg validation error) = (0.2730, 0.2730)
C = 10: (avg train error, avg validation error) = (0.0335, 0.0335)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [35]:
#Fit RBF model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=14, gamma='auto')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.0000
Final Test Error: 0.7393
RBF SVM Performance Metrics with C=100, gamma='auto':
Accuracy: 0.2607
Precision: 0.2690
Recall: 0.2607
F1 Score: 0.2617

Confusion Matrix:
[[ 6  5  6  4 10  7]
 [ 2 20 12  2  7  4]
 [ 7  5  7  4  4 14]
 [ 3 10  4 13 10  6]
 [ 4  8  6  7 12 10]
 [ 5  1  5  4 14  9]]

Classification Report:
              precision    recall  f1-score   support

           1       0.22      0.16      0.18        38
           2       0.41      0.43      0.42        47
           3       0.17      0.17      0.17        41
           4       0.38      0.28      0.33        46
           5       0.21      0.26      0.23        47
           6       0.18      0.24      0.20        38

    accuracy                           0.26       257
   macro avg       0.26      0.25      0.26       257
weighted avg       0.27      0.26      0.26       257



In [36]:
#Selecting best C for RBF SVM Model (gamma=scale)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		scale_svm = SVC(kernel='rbf',C=C, gamma='scale')
		scale_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = scale_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = scale_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.7722, 0.7722)
C = 0.1: (avg train error, avg validation error) = (0.7722, 0.7722)
C = 1.0: (avg train error, avg validation error) = (0.2730, 0.2730)
C = 10: (avg train error, avg validation error) = (0.0335, 0.0335)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [26]:
#Fit RBF model on best C (gamma='scale')
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=C_chosen, gamma='scale')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='scale':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.0000
Final Test Error: 0.7169
RBF SVM Performance Metrics with C=100, gamma='scale':
Accuracy: 0.2831
Precision: 0.3561
Recall: 0.2831
F1 Score: 0.3122

Confusion Matrix:
[[ 7  6  7  2  6 10]
 [ 2 19  7  7  4  8]
 [ 5  5  8  5  7 11]
 [ 2 11  7 13  6  7]
 [ 4  7  4  7 15 10]
 [ 0  0  0  0  0  0]]

Classification Report:
              precision    recall  f1-score   support

           1       0.35      0.18      0.24        38
           2       0.40      0.40      0.40        47
           3       0.24      0.20      0.22        41
           4       0.38      0.28      0.33        46
           5       0.39      0.32      0.35        47
           6       0.00      0.00      0.00         0

    accuracy                           0.28       219
   macro avg       0.29      0.23      0.26       219
weighted avg       0.36      0.28      0.31       219



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

RBF SVM with gamma = scale and gamma = auto performs comparably okay 
RBF SVM performs a lot more better with Linear SVM

DIY SVM

In [37]:
#implement own version of SVM and tweak something?????
#DIY SVM
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define the trigonometric kernel function
def trigonometric_kernel(X, Y, sigma=1.0):
    """
    Trigonometric kernel function as defined in the paper.

    Args:
        X (numpy.ndarray): Matrix of shape (n_samples_X, n_features).
        Y (numpy.ndarray): Matrix of shape (n_samples_Y, n_features).
        sigma (float): Kernel parameter.

    Returns:
        numpy.ndarray: Kernel matrix of shape (n_samples_X, n_samples_Y).
    """
    pairwise_sq_dists = np.sum((X[:, None] - Y[None, :])**2, axis=-1)
    return np.sin(np.pi / 2 + sigma * pairwise_sq_dists)

# Custom kernel wrapper for scikit-learn
class CustomKernelSVM(SVC):
    def __init__(self, sigma=1.0, **kwargs):
        super().__init__(kernel="precomputed", **kwargs)
        self.sigma = sigma

    def fit(self, X, y):
        # Compute the kernel matrix for training data
        self.X_fit_ = X
        K = trigonometric_kernel(X, X, sigma=self.sigma)
        return super().fit(K, y)

    def predict(self, X):
        # Compute the kernel matrix between training and test data
        K = trigonometric_kernel(X, self.X_fit_, sigma=self.sigma)
        return super().predict(K)

# Generate a synthetic dataset for testing
X, y = make_classification(
    n_samples=500, n_features=10, n_classes=3, n_informative=8, n_clusters_per_class=1, random_state=42
)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM with the custom trigonometric kernel
sigma_value = 0.5  # Adjust based on dataset characteristics
svm = CustomKernelSVM(sigma=sigma_value, C=1.0, decision_function_shape="ovr")
svm.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.39      0.41        38
           1       0.39      0.24      0.30        37
           2       0.29      0.48      0.36        25

    accuracy                           0.36       100
   macro avg       0.37      0.37      0.36       100
weighted avg       0.38      0.36      0.36       100



In [38]:
# diy_svm = MultiClassSVM()
# diy_svm.fit(X_train, y_train)
# y_test_pred = diy_svm.predict(y_test)

svm = CustomKernelSVM(sigma=sigma_value, C=1.0, decision_function_shape="ovr")
svm.fit(X_train, y_train)

# Predict and evaluate
y_test_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_test_pred))



final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='scale':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.39      0.41        38
           1       0.39      0.24      0.30        37
           2       0.29      0.48      0.36        25

    accuracy                           0.36       100
   macro avg       0.37      0.37      0.36       100
weighted avg       0.38      0.36      0.36       100

Final Test Error: 0.6400
RBF SVM Performance Metrics with C=100, gamma='scale':
Accuracy: 0.3600
Precision: 0.3763
Recall: 0.3600
F1 Score: 0.3560

Confusion Matrix:
[[15  8 15]
 [14  9 14]
 [ 7  6 12]]

Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.39      0.41        38
           1       0.39      0.24      0.30        37
           2       0.29      0.48      0.36        25

    accuracy                           0.36       100
   macro avg       0.37      0.37      0.36       100
weighted avg       0.38      0.36      0.36